# Decision trees




In this section we cover the decision tree methods. We divided it into three parts:
- univariate methods,
- multivariate methods,
- quality metrics.

The customer segmentation shown in the lectures looks as following:


| Location|Category   | Gender | Product review  | Decision |
|---------|-----------|--------|-----------------|--------|
| Berlin | Furniture  | Male   | Checked Reviews | Bought |
| London | Furniture  | Male   | Checked Reviews | Bought |
| Berlin | Furniture  | Female | Checked Reviews | Did not bought |
| Berlin | Textile    | Female | Checked Reviews | Bought |
| London | Electronics| Male   | Checked Reviews | Did not bough |
| London | Textile    | Female | Checked Reviews | Did not bough |
| Paris  | Textile    | Male   | Did not checked | Bought |
| Berlin | Electronics| Male   | Checked Reviews | Bought |
| Paris  | Electronics| Male   | Did not checked | Bought |
| London | Electronics| Female | Checked Reviews | Bought |
| Paris  | Furniture  | Female | Did not checked | Bought |
| Berlin | Textile    | Female | Did not checked | Bought |
| London | Electronics| Female | Did not checked | Did not bough |
| London | Furniture  | Female | Checked Reviews | Did not bough |
| London | Textile    | Female | Did not checked | Did not bough |

In [2389]:
import numpy

labels = [1,1,-1,1,-1,-1,1,1,1,1,1,1,-1,-1,-1]
X = [[1,1,2,2],[2,1,2,2],[1,1,1,2],[1,2,1,2],[2,3,2,2],
                [2,2,1,2],[3,2,2,1],[1,3,2,2],[3,3,2,1],[2,3,1,2],
                [3,1,1,1],[1,2,1,1],[2,3,1,1],[2,1,1,2],[2,2,1,1]]

## Univariate methods

In this section we cover two methods:
- CART,
- C4.5.

There are many more univariate methods. 

### CART

In [2390]:
class BinaryLeaf:

    def __init__(self, elements, labels):
        self.L = None
        self.R = None
        self.elements = elements
        self.labels = labels
        self.completed = False

    def set_R(self, Rleaf):
        self.R = Rleaf

    def set_L(self, Lleaf):
        self.L = Lleaf

    def set_elements(self, elements):
        self.elements = elements

    def get_elements(self):
        return self.elements

    def set_p(self, threshold):
        self.p = threshold

    def get_L(self):
        return self.L

    def get_R(self):
        return self.R

    def set_completed(self):
        self.completed = True

    def is_completed(self):
        return self.completed

    def get_labels(self):
        return self.labels


In [2391]:
class TreeUtils:

    def __init__(self, root):
        self.root_node = root

    def get_unique_labels(self, labels):
        return numpy.unique(numpy.array(labels)).tolist()

    def get_unique_values(self, elements):
        features_number = len(elements[0])
        unique = []
        for i in range(features_number):
            features_list = []
            for j in range(len(elements)):
                features_list.append(elements[j][i])
            unique.append(numpy.unique(numpy.array(features_list)))
        return unique

    def is_binary_leaf_completed(self, node):
        if node.is_completed():
            if node.get_L() != None and not node.get_L().is_completed():
                return node.get_L()
            elif node.get_R() != None and not node.get_R().is_completed():
                return node.get_R()
            elif node.get_L() == None and node.get_R() == None:
                return None
            elif node.get_L().is_completed() or node.get_R().is_completed():
                new_node = self.is_binary_leaf_completed(node.get_L())
                if new_node == None:
                    return self.is_binary_leaf_completed(node.get_R())
                else:
                    return new_node
            else:
                return None
        return node

    def find_binary_leaf_not_completed(self):
        return self.is_binary_leaf_completed(self.root_node)

    def is_leaf_completed(self, node):
        if node.is_completed():
            child_nodes = node.get_child_leafs()
            if len(child_nodes) == 0:
                return None
            is_child_to_return = False
            for i in range(len(child_nodes)):
                if not child_nodes[i].is_completed():
                    return child_nodes[i]
                else:
                    new_node = self.is_leaf_completed(child_nodes[i])
                    if new_node != None:
                        is_child_to_return=True
            if is_child_to_return:
                return new_node
        return node

    def find_leaf_not_completed(self):
        return self.is_leaf_completed(self.root_node)
    
    def print_tree(self, root_node):
        
        leafs_exists=True
        level = 0

        print("Level: " + str(level))
        print(str(root_node.get_elements()))
        print(str(root_node.get_labels()))

        current_nodes=[]
        childs = root_node.get_child_leafs()
        for i in range(len(childs)):
            current_nodes.append([root_node,childs[i]])

        while leafs_exists:
            level = level + 1
            print("Level: " + str(level))
            new_nodes = []
            for i in range(len(current_nodes)):
                print("ancestor: "+ str(current_nodes[i][0].get_elements()) + " " + str(current_nodes[i][0].get_labels()))
                print("child1: "   + str(current_nodes[i][1].get_elements()) + " " + str(current_nodes[i][1].get_labels()))
                print("")
                if current_nodes[i][1].get_child_leafs() != None:
                    childs = current_nodes[i][1].get_child_leafs()
                    for j in range(len(childs)):
                        new_nodes.append([current_nodes[i][1],childs[j]])
            if len(new_nodes) == 0:
                leafs_exists = False
            else:
                current_nodes = new_nodes
                
                
    def write_tree(self, root_node):
        import pydot 
        graph = pydot.Dot(graph_type='graph')
        features = [{1: "Berlin", 2: "London", 3:"Paris"}, 
                    {1: "Furniture", 2: "Textile", 3: "Electronics"}, 
                    {1: "Female", 2: "Male"}, 
                    {1: "Didn't checked", 2: "Checked review"}, 
                    {1: "Bought", -1:"Didn't bought"}]
        
        leafs_exists=True
        level = 0

        current_nodes=[]
        childs = root_node.get_child_leafs()
        for i in range(len(childs)):
            current_nodes.append([root_node,childs[i]])
        counter = 0
        while leafs_exists:
            level = level + 1
            new_nodes = []
            for i in range(len(current_nodes)):
                if level == 1:
                    graph.add_node(pydot.Node(str(current_nodes[i][0].get_elements()), label='root'))
                    if i == 2:
                        graph.add_node(pydot.Node(str(current_nodes[i][1].get_elements()), label='Paris'))
                        for result in current_nodes[i][1].get_labels():
                            color = "red" if result == -1 else "green"
                            graph.add_node(pydot.Node(str(result + 189), label=str(result), color=color))
                            edge = pydot.Edge(str(current_nodes[i][1].get_elements()), str(result + 189))
                            graph.add_edge(edge)
                elif level == 2:
                    for l in current_nodes[i][0].get_elements():
                        city = "Berlin" if l[0] == 1 else "London"
                    graph.add_node(pydot.Node(str(current_nodes[i][0].get_elements()), label=city))
                edge = pydot.Edge(str(current_nodes[i][0].get_elements()), str(current_nodes[i][1].get_elements()))
                graph.add_edge(edge)
                if current_nodes[i][1].get_child_leafs() != None:
                    childs = current_nodes[i][1].get_child_leafs()
                    for j in range(len(childs)):
                        new_nodes.append([current_nodes[i][1],childs[j]])
                if level == 2:
                    for result in current_nodes[i][1].get_labels():
                        color = "red" if result == -1 else "green"
                        l = current_nodes[i][1].get_elements()
                        label1 = features[1].get(l[0][1])
                        label2 =''
                        for k in range(2):
                            label2 += " " + features[k+2].get(l[0][k+2])
                        graph.add_node(pydot.Node(str(result + counter), label=features[4].get(result), color=color))
                        graph.add_node(pydot.Node(str(current_nodes[i][1].get_elements()), label=label1))
                        edge = pydot.Edge(str(current_nodes[i][1].get_elements()), str(result + counter), label=label2)
                        counter += 1
                        graph.add_edge(edge)
                
            if len(new_nodes) == 0:
                leafs_exists = False
            else:
                current_nodes = new_nodes

        graph.write_png('decision_tree.png')

    def print_binary_tree(self, root_node):
    
        leafs_exists=True
        level = 0

        print("Level: " + str(level))
        print(str(root_node.get_elements()))
        print(str(root_node.get_labels()))

        current_nodes=[]
        if root_node.get_L() != None:
            current_nodes.append([root_node, root_node.get_L()])
        if root_node.get_R() != None:
            current_nodes.append([root_node, root_node.get_R()])

        while leafs_exists:
            level = level + 1
            print("Level: " + str(level))
            new_nodes = []
            for i in range(len(current_nodes)):
                print("ancestor: "+ str(current_nodes[i][0].get_elements()) + " " + str(current_nodes[i][0].get_labels()))
                print("child1: "   + str(current_nodes[i][1].get_elements()) + " " + str(current_nodes[i][1].get_labels()))
                print("")
                if current_nodes[i][1].get_L() != None:
                    new_nodes.append([current_nodes[i][1], current_nodes[i][1].get_L()])
                if current_nodes[i][1].get_R() != None:
                    new_nodes.append([current_nodes[i][1], current_nodes[i][1].get_R()])
            if len(new_nodes) == 0:
                leafs_exists = False
            else:
                current_nodes = new_nodes    

In [2392]:
root = BinaryLeaf(X, labels)
utils = TreeUtils(root)
current_node = root
labels_count = 2

In [2393]:
import copy

def get_split_candidates(unique_values):
    split_list = []
    for i in range(len(unique_values)):
        current_list=[]
        temp_list=copy.deepcopy(unique_values)
        current_list.append(temp_list[i])
        del temp_list[i]
        current_list.append(temp_list)
        split_list.append(current_list)
    return split_list

In [2394]:
def get_number_of_labels_for_value(elements, column_id, label):
    count = 0
    if not isinstance(elements, list):
        elements_list = [elements]
    else:
        elements_list = elements

    column_elements = get_node_elements_column(column_id)

    for i in range(len(elements_list)):
        for j in range(len(column_elements)):
            if column_elements[j] == elements_list[i]:
                if current_node.labels[j] == label:
                    count = count + 1
    return count

In [2395]:
def get_node_elements_column(column_id):
    return numpy.array(current_node.elements)[...,column_id].tolist()

In [2396]:
def count_number_of_elements(elements, column_id):
    count = 0
    if isinstance(elements, list):
        column_elements = get_node_elements_column(column_id)
        for i in range(len(elements)):
            count = count + column_elements.count(elements[i])
    else:
        count = count + get_node_elements_column(column_id).count(elements)
    return count

In [2424]:
import math

def calculate_omega(elements, column_id):
    global utils
    global current_node
    global labels_count
    t_l = count_number_of_elements(elements[0], column_id)
    t_r = count_number_of_elements(elements[1], column_id)
    p_l = t_l * 1.0 / len(current_node.elements) * 1.0
    p_r = t_r * 1.0 / len(current_node.elements) * 1.0


    sum_p = 0
    labels = utils.get_unique_labels(current_node.labels)
    for i in range(labels_count):
        p_class_t_l = (get_number_of_labels_for_value(elements[0], column_id, labels[i])*1.0)/(count_number_of_elements(elements[0], column_id)*1.0)
        p_class_t_r = (get_number_of_labels_for_value(elements[1], column_id, labels[i])*1.0)/(count_number_of_elements(elements[1], column_id)*1.0)
        sum_p = sum_p + math.fabs(p_class_t_l-p_class_t_r)
    return 2.0*p_l*p_r*sum_p

In [2425]:
def check_completed(labels, elements):
    global utils
    ratio = len(utils.get_unique_labels(labels))
    if ratio == 1:
        return True
    elements=sorted(elements)
    duplicated = [elements[i] for i in range(len(elements)) if i == 0 or elements[i] != elements[i-1]]
    if len(duplicated) == 1:
        return True
    return False

In [2426]:
def split_node(value, split_id):
    global current_node
    left_leaf = []
    left_leaf_labels = []
    right_leaf = []
    right_leaf_labels = []
    for i in range(len(current_node.elements)):
        if current_node.elements[i][split_id] == value:
            left_leaf.append(current_node.elements[i])
            left_leaf_labels.append(current_node.labels[i])
        else:
            right_leaf.append(current_node.elements[i])
            right_leaf_labels.append(current_node.labels[i])
    current_node.set_L(BinaryLeaf(left_leaf, left_leaf_labels))
    current_node.set_R(BinaryLeaf(right_leaf, right_leaf_labels))
    current_node.set_completed()
    if check_completed(left_leaf_labels, left_leaf):
        current_node.L.set_completed()
    if check_completed(right_leaf_labels, right_leaf):
        current_node.R.set_completed()

In [2427]:
def get_current_node():
    global utils
    return utils.find_binary_leaf_not_completed()

In [2456]:
def build():
    global current_node
    global utils
    stop_criterion = False
    while stop_criterion == False:
        unique_values = utils.get_unique_values(current_node.get_elements())
        max_unique_id = 0
        max_split_id = 0
        max_value = None
        for i in range(len(unique_values)):
            if len(unique_values[i]) == 1:
                continue
            split_candidates = get_split_candidates(unique_values[i].tolist())
            for j in range(len(split_candidates)):
                current_value = calculate_omega(split_candidates[j],i)
                if max_value == None:
                    max_unique_id = i
                    max_split_id = j
                    max_value = current_value                    
                elif max_value < current_value:
                    max_unique_id = i
                    max_split_id = j
                    max_value = current_value
                print(max_unique_id)
        split_node(unique_values[max_unique_id][max_split_id], max_unique_id)
        new_node = get_current_node()
        if new_node != None:
            current_node = new_node
        else:
            stop_criterion = True

In [2457]:
build()

2
2


In [2458]:
def show_tree():
    global root
    global utils
    utils.print_binary_tree(root)

In [2447]:
show_tree()

Level: 0
[[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]]
[1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
Level: 1
ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child1: [[2, 1, 2, 2], [2, 3, 2, 2], [2, 2, 1, 2], [2, 3, 1, 2], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, -1, -1, 1, -1, -1, -1]

ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child1: [[1, 1, 2, 2], [1, 1, 1, 2], [

#### Exercise 1: Rewrite the CART method to use Gini index as shown in the lecture

Use the following equation:
\begin{equation}
I_{G}(X)=1-\sum_{i=1}^{m}p^{2}_{i},
\end{equation}
and
\begin{equation}
I_{G}(\text{feature})=1-\sum_{i=1}^{n}p_{i}*I_{G}(X_{i}),
\end{equation}



In [2378]:
def calculate_omega(elements, column_id):
    global utils
    global current_node
    global labels_count
    t_l = count_number_of_elements(elements[0], column_id)
    t_r = count_number_of_elements(elements[1], column_id)
    p_l = t_l * 1.0 / len(current_node.elements) * 1.0
    p_r = t_r * 1.0 / len(current_node.elements) * 1.0
    
    sum_p = 0
    labels = utils.get_unique_labels(current_node.labels)
    for i in range(labels_count):
        p_class_t_l = (get_number_of_labels_for_value(elements[0], column_id, labels[i])*1.0)/(count_number_of_elements(elements[0], column_id)*1.0)
        p_class_t_r = (get_number_of_labels_for_value(elements[1], column_id, labels[i])*1.0)/(count_number_of_elements(elements[1], column_id)*1.0)
        sum_p = sum_p + math.fabs(p_class_t_l-p_class_t_r)
    return 2.0*p_l*p_r*sum_p



### C4.5

In [2379]:
class Leaf:

    def __init__(self, elements, labels):
        self.child_leafs = []
        self.elements = elements
        self.labels = labels
        self.completed = False

    def get_elements(self):
        return self.elements

    def set_child_leafs(self, new_leafs):
        self.child_leafs = new_leafs

    def set_completed(self):
        self.completed = True

    def is_completed(self):
        return self.completed

    def get_labels(self):
        return self.labels

    def get_child_leafs(self):
        return self.child_leafs

In [2380]:
root = Leaf(X, labels)
utils = TreeUtils(root)
current_node = root
labels_count = labels_count

In [2381]:
def get_current_node():
    global utils
    return utils.find_leaf_not_completed()

In [2382]:
def split(split_values, column_id):
    global current_node
    new_leafs = []
    for i in range(len(split_values)):
        indices = numpy.where(numpy.array(current_node.get_elements())[..., column_id].tolist() == split_values[i])
        new_leaf_elements = numpy.array(current_node.get_elements())[indices].tolist()
        new_leaf_labels   = numpy.array(current_node.get_labels())[indices].tolist()
        new_leaf = Leaf(new_leaf_elements,new_leaf_labels)
        if len(numpy.unique(new_leaf_labels)) == 1:
            new_leaf.set_completed()
        new_leafs.append(new_leaf)
    current_node.set_child_leafs(new_leafs)
    current_node.set_completed()

In [2383]:
class Entropy:

    def calculate_entropy(self, labels):
        unique_labels, labels_count = numpy.unique(labels, return_counts=True)
        entropy = 0
        size = len(labels)
        for i in range(len(unique_labels)):
            if labels_count[i] > 0:
                log2=math.log((labels_count[i]*1.0)/(size*1.0),2)
            else:
                log2=0.0
            entropy=entropy-1.0*((labels_count[i]*1.0)/size)*log2
        return entropy

    def calculate_split_candidate_entropy(self, full_entropy, labels, elements, unique_labels, unique_elements, iter):
        split_entropy=0
        for i in range(len(unique_elements)):
            indices = numpy.where(numpy.array(elements)[..., iter].tolist() == unique_elements[i])
            unique_size = len(indices[0].tolist())
            filtered_labels = numpy.array(labels)[indices]
            for j in range(len(unique_labels)):
                labels_count = filtered_labels.tolist().count(unique_labels[j])
                if labels_count > 0:
                    log2=math.log((labels_count*1.0)/(unique_size*1.0),2)
                else:
                    log2=0.0
                split_entropy = split_entropy -1.0*((labels_count*1.0)/unique_size*1.0)*log2*unique_size*1.0/len(elements)*1.0
        return (full_entropy-split_entropy)


In [2384]:
def build():
    global root
    global current_node
    stop_criterion = False
    current_node = root
    entropy = Entropy()
    unique_labels = utils.get_unique_labels(root.get_labels())
    while stop_criterion == False:
        unique_values = utils.get_unique_values(current_node.get_elements())
        full_entropy = entropy.calculate_entropy(current_node.get_labels())
        max_entropy_id = 0
        max_entropy_value = 0
        for i in range(len(unique_values)):
            split_entropy = entropy.calculate_split_candidate_entropy(full_entropy, current_node.get_labels(), current_node.get_elements(), unique_labels, unique_values[i], i)
            if split_entropy > max_entropy_value:
                max_entropy_id = i
                max_entropy_value = split_entropy
        split(unique_values[max_entropy_id], max_entropy_id)
        new_node = get_current_node()
        if new_node != None:
            current_node = new_node
        else:
            stop_criterion = True

In [2385]:
build()

In [2386]:
def show_tree():
    global utils
    global root
    utils.print_tree(root)
    utils.write_tree(root)

In [2387]:
show_tree()

Level: 0
[[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]]
[1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
Level: 1
ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child1: [[1, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [1, 3, 2, 2], [1, 2, 1, 1]] [1, -1, 1, 1, 1]

ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child1: [[2, 1, 2, 2], [2, 3, 2, 2], [2, 2, 1, 2], [2, 3, 1, 2], [2, 3, 1, 1

# Exercise 2: Use pydot do draw the tree for C4.5 example

In [2388]:
#write_tree function is created in TreeUtils class. Also used it in show_tree() function